In [26]:
import time

from ib.opt import message
import numpy as np
import pandas as pd

import contracts
from settings import TWS_CONNECTION, company

In [49]:
class GetOptionsPrices():
    contract = None
    right = None
    prices_put = pd.DataFrame()
    prices_call = pd.DataFrame()
    prices_msg = pd.DataFrame()
   
    def __init__(self, company, expiration, strike):
        self.company = company
        self.expiration = expiration
        self.strike = strike

    def get_prices(self):
        TWS_CONNECTION.register(self.msg_handler, message.historicalData)
        TWS_CONNECTION.register(self.error_handler, message.Error)
#         TWS_CONNECTION.registerAll(print)
        self.get_put_prices()
        prices_put = self.prices_msg.set_index('Datetime')
        self.prices_msg = pd.DataFrame()
        self.get_call_prices()
        prices_call = self.prices_msg.set_index('Datetime')
        # Merge PUT & CALL into one DateFrame
        return pd.concat([prices_call, prices_put], axis=1, sort=False)
       
    def get_put_prices(self):
        TWS_CONNECTION.connect()
        self.right = 'P' # for the column name
        self.contract = contracts.PutOption(
            self.company,
            self.expiration,
            self.strike
        )
        self.request_prices()
        time.sleep(2)
        TWS_CONNECTION.disconnect()

    def get_call_prices(self):
        TWS_CONNECTION.connect()
        self.right = 'C' # for the column name
        self.contract = contracts.CallOption(
            self.company,
            self.expiration,
            self.strike
        )
        self.request_prices()
        time.sleep(2)
        TWS_CONNECTION.disconnect()
 
    def request_prices(self):
        TWS_CONNECTION.reqHistoricalData(
            1,
            self.contract,
            '',
            '1 W',
            '30 mins',
            "TRADES",
            1,
            1
        )
        
    def msg_handler(self, msg):
        """
        Function get message from TWS and put it into pandas DataFrame object
        :param msg: message from TWS
        :return: dataframe w/ historical prices
        """
        new_row = pd.DataFrame(
            {'Datetime': msg.date,
            f'{self.right}-{self.expiration}-{self.strike}': msg.close},
            index=[1])
        self.prices_msg = self.prices_msg.append(new_row, ignore_index=True)
    
    def error_handler(self, msg):
        print(type(msg), msg)        

In [48]:
a = {
	'date': '20191127  21:30:00',
	'P-2019-12-06': 0.37,
}
b = {
	'date': '20191128  21:30:00',
	'P-2019-12-06': 1.37,
}
c = {
	'date': '20191128  21:30:00',
	'C-2019-12-06': 1.37,
}
test = pd.DataFrame()
a = pd.DataFrame(a, index = [0]).set_index('date')
b = pd.DataFrame(b, index = [0]).set_index('date')
test = test.append(a)
test = test.append(b)
# print(test)
c = pd.DataFrame(c, index = [0]).set_index('date')
# print(test)
# print(c)
# test = test.merge(c)#, left_index=True, right_index=True)
test = pd.concat([test, c], axis=1, sort=False)
print(test)

                    P-2019-12-06  C-2019-12-06
20191127  21:30:00          0.37           NaN
20191128  21:30:00          1.37          1.37


In [36]:
prices = GetOptionsPrices(
    company,
    '20191206',
    300
).get_prices()

<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<class 'ib.opt.message.Error'> <error id=-1, errorCode=2104, errorMsg=Market data farm connection

In [37]:
prices

,C-20191206-300,P-20191206-300
Datetime,,
20191125 17:30:00,43.05,0.44
20191125 18:00:00,43.05,0.46
20191125 18:30:00,38.95,0.53
20191125 19:00:00,38.95,0.54
20191125 19:30:00,38.95,0.52
20191125 20:00:00,38.95,0.46
20191125 20:30:00,38.95,0.48
20191125 21:00:00,38.95,0.50
20191125 21:30:00,38.95,0.50


In [2]:
# Look for all strikes
from ib.ext.ContractDetails import ContractDetails
from ib.ext.EClientSocket import EClientSocket


def get_option_chain():
    contract = contracts.CallOption(
        company,
        None,
        None
    )

    msgs = []

    def msg_handler(msg):
        print(type(msg), msg)
    #     print(msg.m_summary)
        msgs.append(msg)

    # TWS_CONNECTION.registerAll(print)
    TWS_CONNECTION.register(msg_handler, 'ContractDetails')
    TWS_CONNECTION.register(msg_handler, 'ContractDetailsEnd')
    TWS_CONNECTION.connect()
    EClientSocket.reqContractDetails(TWS_CONNECTION, 1, contract)

    time.sleep(10)
    TWS_CONNECTION.disconnect()
    i = 1
    for msg in msgs:
        try:
            print(i, msg.contractDetails.m_summary.m_strike, msg.contractDetails.m_summary.m_expiry)
            i += 1
        except AttributeError:
            print(msg)

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd0278>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd0588>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd0898>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd0ba8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd0e80>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110bd8198>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c672e8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c675c0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c67898>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c67b70>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c67e48>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c6a160>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c992b0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c99588>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c99860>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c99b38>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c99e10>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110c9a128>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc46d8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc49b0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc4c88>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc4f60>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc90f0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cc9550>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cfc7f0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cfcb00>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110cfc0b8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110ce00b8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110ce0390>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110ce0668>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

...

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f24828>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f24a58>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f24cf8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f24f60>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f1a208>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f1a470>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5f8d0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5fb00>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5fda0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f64048>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f642b0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f64518>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f95c18>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f95ef0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5a0f0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5a390>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5a5f8>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f5a860>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110fc4be0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110fc4e10>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f940f0>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f94358>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f94588>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDetails object at 0x110f94828>>
<class 'ib.opt.message.ContractDetails'> <contractDetails reqId=1, contractDetails=<ib.ext.ContractDetails.ContractDet

True

1 175.0 20200117
2 180.0 20200117
3 185.0 20200117
4 190.0 20200117
5 195.0 20200117
6 200.0 20200117
7 210.0 20200117
8 220.0 20200117
9 230.0 20200117
10 240.0 20200117
11 250.0 20200117
12 255.0 20200117
13 260.0 20200117
14 265.0 20200117
15 270.0 20200117
16 275.0 20200117
17 280.0 20200117
18 285.0 20200117
19 290.0 20200117
20 295.0 20200117
21 300.0 20200117
22 305.0 20200117
23 310.0 20200117
24 315.0 20200117
25 320.0 20200117
26 325.0 20200117
27 330.0 20200117
28 335.0 20200117
29 340.0 20200117
30 350.0 20200117
31 360.0 20200117
32 370.0 20200117
33 380.0 20200117
34 390.0 20200117
35 400.0 20200117
36 410.0 20200117
37 420.0 20200117
38 430.0 20200117
39 440.0 20200117
40 450.0 20200117
41 100.0 20200117
42 105.0 20200117
43 110.0 20200117
44 115.0 20200117
45 120.0 20200117
46 125.0 20200117
47 130.0 20200117
48 135.0 20200117
49 140.0 20200117
50 145.0 20200117
51 150.0 20200117
52 155.0 20200117
53 160.0 20200117
54 165.0 20200117
55 170.0 20200117
56 50.0 20200117
57

638 110.0 20210917
639 115.0 20210917
640 120.0 20210917
641 125.0 20210917
642 130.0 20210917
643 135.0 20210917
644 140.0 20210917
645 145.0 20210917
646 150.0 20210917
647 155.0 20210917
648 15.0 20210917
649 160.0 20210917
650 165.0 20210917
651 170.0 20210917
652 175.0 20210917
653 180.0 20210917
654 185.0 20210917
655 190.0 20210917
656 195.0 20210917
657 20.0 20210917
658 25.0 20210917
659 280.0 20210917
660 290.0 20210917
661 300.0 20210917
662 30.0 20210917
663 310.0 20210917
664 5.0 20200619
665 320.0 20210917
666 5.0 20210115
667 330.0 20210917
668 340.0 20210917
669 35.0 20210917
670 40.0 20210917
671 45.0 20210917
672 50.0 20210917
673 55.0 20210917
674 60.0 20210917
675 65.0 20210917
676 70.0 20210917
677 75.0 20210917
678 80.0 20210917
679 85.0 20210917
680 90.0 20210917
681 95.0 20210917
682 10.0 20210319
683 5.0 20210319
684 5.0 20210917
685 10.0 20210917
686 5.0 20220121
687 10.0 20220121
688 200.0 20191129
689 205.0 20191129
690 210.0 20191129
691 215.0 20191129
692 

1137 600.0 20220121
1138 610.0 20220121
1139 620.0 20220121
1140 630.0 20220121
1141 640.0 20220121
1142 287.5 20191129
1143 292.5 20191129
1144 297.5 20191129
1145 302.5 20191129
1146 307.5 20191129
1147 312.5 20191129
1148 317.5 20191129
1149 322.5 20191129
1150 327.5 20191129
1151 332.5 20191129
1152 337.5 20191129
1153 342.5 20191129
1154 347.5 20191129
1155 205.0 20191213
1156 210.0 20191213
1157 215.0 20191213
1158 220.0 20191213
1159 225.0 20191213
1160 230.0 20191213
1161 235.0 20191213
1162 240.0 20191213
1163 200.0 20191227
1164 500.0 20200320
1165 245.0 20191213
1166 250.0 20191213
1167 255.0 20191213
1168 260.0 20191213
1169 265.0 20191213
1170 270.0 20191213
1171 275.0 20191213
1172 277.5 20191213
1173 280.0 20191213
1174 282.5 20191213
1175 285.0 20191213
1176 287.5 20191213
1177 290.0 20191213
1178 292.5 20191213
1179 295.0 20191213
1180 297.5 20191213
1181 300.0 20191213
1182 302.5 20191213
1183 305.0 20191213
1184 307.5 20191213
1185 310.0 20191213
1186 312.5 20191213


In [24]:
! git commit -a -m 'Bbbback!'
! git push

[version_2 99e08d3] Bbbback!
 2 files changed, 60 insertions(+), 40 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 648 bytes | 648.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/SVGalich/tws_trader.git
   e3ca501..99e08d3  version_2 -> version_2
